In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers
!pip install accelerate
!pip install transformers

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, DiffusionPipeline
import torch
from diffusers import UniPCMultistepScheduler
from diffusers.utils import load_image
import cv2
from PIL import Image
import numpy as np
from transformers import pipeline,SamModel, SamProcessor
import torch 

## Sam Model for masking 

In [ ]:
generator = pipeline(
    task="mask-generation",
    model="facebook/sam-vit-base",
    processor="facebook/sam-vit-base",
    device=0,
    points_per_batch=256
)

In [ ]:
image_path = "/kaggle/input/jewelry/OSN23015_3.jpg"
image = Image.open(image_path)
image

In [ ]:
outputs = generator(image_path, points_per_batch = 256)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


# Extract the first mask from the list
first_mask = outputs['masks'][0]
# Plot the mask
plt.figure(figsize=(6,6))
plt.imshow(first_mask, cmap='gray')  # Use a gray colormap to visualize the mask
plt.axis('off')  # Hide the axes
# Save the mask as a JPG image
plt.savefig("mask_image.jpg", format='jpg', bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
save_path=r"/kaggle/working/"
first_mask.save(save_path)

## Inpanting 

import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid

pipe = AutoPipelineForInpainting.from_pretrained(
    "diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()
pipe.to("cuda")

In [ ]:
first_mask="/kaggle/working/mask_image.jpg"
# image_path = "/kaggle/input/jewelry/OSN23015_3.jpg"
mask = Image.open(first_mask)
mask

In [ ]:
prompt = '''Generate black color background'''
neg_prompt = "Inconsistent colors, distorted image, clumsy design, chaotic elements, mismatched aesthetics, unclean and fuzzy patterns, same color scheme, ugly patterns, varied artistic style"

In [ ]:
# Run the inpainting pipeline
output = pipe(prompt=prompt, negative_prompt = neg_prompt,image=image, mask_image=mask)
result_image = output.images[0]
result_image